# Comparing Network Construction Algorithms

This notebook compares the four main network construction algorithms available in PyPopART:
1. **MST** - Minimum Spanning Tree
2. **MSN** - Minimum Spanning Network
3. **TCS** - Statistical Parsimony
4. **MJN** - Median-Joining Network

We'll construct networks using the same data and compare their properties.

In [ ]:
import matplotlib.pyplot as plt
from pypopart.io import load_alignment
from pypopart.core.distance import DistanceCalculator
from pypopart.core.condensation import condense_alignment
from pypopart.algorithms import MSTAlgorithm, MSNAlgorithm, TCSAlgorithm, MJNAlgorithm
from pypopart.stats import calculate_network_metrics
from pypopart.visualization import StaticNetworkPlotter

## Prepare Data

In [ ]:
# Load and prepare data
alignment = load_alignment('../../data/examples/sample.fasta')
calculator = DistanceCalculator(method='hamming')
dist_matrix = calculator.calculate_matrix(alignment)

print(f"Loaded alignment with {len(alignment)} sequences")
print(f"Alignment length: {alignment.length} bp")

## 1. Minimum Spanning Tree (MST)

The simplest algorithm - creates a tree with minimum total edge weight.
- No reticulations (cycles)
- Connects all haplotypes
- Deterministic given tie-breaking rules

In [ ]:
mst_algo = MSTAlgorithm()
mst_network = mst_algo.construct_network(alignment, dist_matrix)

mst_metrics = calculate_network_metrics(mst_network)
print("MST Network:")
print(f"  Nodes: {mst_network.num_nodes}")
print(f"  Edges: {mst_network.num_edges}")
print(f"  Reticulation index: {mst_metrics.reticulation_index:.4f}")

## 2. Minimum Spanning Network (MSN)

Extends MST by adding alternative edges at equal distance.
- Shows alternative evolutionary paths
- May contain reticulations
- More complex than MST

In [ ]:
msn_algo = MSNAlgorithm()
msn_network = msn_algo.construct_network(alignment, dist_matrix)

msn_metrics = calculate_network_metrics(msn_network)
print("MSN Network:")
print(f"  Nodes: {msn_network.num_nodes}")
print(f"  Edges: {msn_network.num_edges}")
print(f"  Reticulation index: {msn_metrics.reticulation_index:.4f}")

## 3. TCS (Statistical Parsimony)

Connects haplotypes within a parsimony probability limit.
- Statistically justified connections
- May result in disconnected components
- Connection limit based on sequence length and probability threshold

In [ ]:
tcs_algo = TCSAlgorithm(connection_limit=0.95)
tcs_network = tcs_algo.construct_network(alignment, dist_matrix)

tcs_metrics = calculate_network_metrics(tcs_network)
print("TCS Network:")
print(f"  Nodes: {tcs_network.num_nodes}")
print(f"  Edges: {tcs_network.num_edges}")
print(f"  Connection limit: {tcs_algo.get_connection_limit(alignment.length)} mutations")
print(f"  Reticulation index: {tcs_metrics.reticulation_index:.4f}")

## 4. Median-Joining Network (MJN)

Infers ancestral/median sequences and creates reticulate network.
- Can infer unsampled ancestral haplotypes
- Most complex algorithm
- Epsilon parameter controls complexity

In [ ]:
mjn_algo = MJNAlgorithm(epsilon=0)
mjn_network = mjn_algo.construct_network(alignment, dist_matrix)

mjn_metrics = calculate_network_metrics(mjn_network)
n_medians = len([n for n in mjn_network.graph.nodes() if mjn_network.graph.nodes[n].get('is_median', False)])

print("MJN Network:")
print(f"  Nodes: {mjn_network.num_nodes}")
print(f"  Edges: {mjn_network.num_edges}")
print(f"  Median vectors: {n_medians}")
print(f"  Reticulation index: {mjn_metrics.reticulation_index:.4f}")

## Visual Comparison

Let's visualize all four networks side by side.

In [ ]:
# Create figure with 4 subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 14))
fig.suptitle('Network Algorithm Comparison', fontsize=16, fontweight='bold')

networks = [
    (mst_network, 'MST - Minimum Spanning Tree', axes[0, 0]),
    (msn_network, 'MSN - Minimum Spanning Network', axes[0, 1]),
    (tcs_network, 'TCS - Statistical Parsimony', axes[1, 0]),
    (mjn_network, 'MJN - Median-Joining Network', axes[1, 1])
]

for network, title, subplot_ax in networks:
    plotter = StaticNetworkPlotter(network)
    # Create plot and get the figure and axis
    temp_fig, temp_ax = plotter.plot(
        layout_algorithm='spring',
        show_labels=True,
        title=title,
        figsize=(7, 6)
    )
    # Close the temporary figure since we're using subplots
    plt.close(temp_fig)

# Note: For proper subplot visualization, it's better to plot each network separately
# or use the individual plotter objects. For now, create individual plots:

print("\nTo create individual plots:")
for i, (network, title, _) in enumerate(networks):
    plotter = StaticNetworkPlotter(network)
    fig, ax = plotter.plot(
        layout_algorithm='spring',
        show_labels=True,
        title=title,
        figsize=(8, 7)
    )
    filename = f'network_{["mst", "msn", "tcs", "mjn"][i]}.png'
    fig.savefig(filename, dpi=150, bbox_inches='tight')
    plt.close(fig)
    print(f"  Saved {filename}")

## Summary Statistics Comparison

In [ ]:
import pandas as pd

# Collect statistics
comparison_data = {
    'Algorithm': ['MST', 'MSN', 'TCS', 'MJN'],
    'Nodes': [
        mst_network.num_nodes,
        msn_network.num_nodes,
        tcs_network.num_nodes,
        mjn_network.num_nodes
    ],
    'Edges': [
        mst_network.num_edges,
        msn_network.num_edges,
        tcs_network.num_edges,
        mjn_network.num_edges
    ],
    'Reticulation': [
        mst_metrics.reticulation_index,
        msn_metrics.reticulation_index,
        tcs_metrics.reticulation_index,
        mjn_metrics.reticulation_index
    ],
    'Diameter': [
        mst_metrics.diameter,
        msn_metrics.diameter,
        tcs_metrics.diameter,
        mjn_metrics.diameter
    ]
}

df = pd.DataFrame(comparison_data)
print("\nComparison Table:")
print(df.to_string(index=False))

## When to Use Each Algorithm

### MST (Minimum Spanning Tree)
**Use when:**
- You want the simplest possible network
- You're interested in a strict hierarchical relationship
- Computational speed is a priority
- You want a tree structure without reticulations

**Avoid when:**
- Recombination or horizontal gene transfer is likely
- You need to show alternative evolutionary paths

### MSN (Minimum Spanning Network)
**Use when:**
- You want to show alternative evolutionary pathways
- Multiple paths may exist at the same genetic distance
- You need more information than MST but want to stay simple

**Avoid when:**
- You need ancestral sequence inference
- You want strict statistical justification for connections

### TCS (Statistical Parsimony)
**Use when:**
- You want statistically justified connections
- You're working with closely related sequences
- You need to identify separate networks (disconnected components)
- Publication requires statistical support for connections

**Avoid when:**
- Sequences are very divergent (may result in disconnected networks)
- You need to infer ancestral sequences

### MJN (Median-Joining Network)
**Use when:**
- You want to infer ancestral/unsampled haplotypes
- Population has complex reticulate history
- You need the most comprehensive network
- Recombination or incomplete lineage sorting is possible

**Avoid when:**
- You want a simple, easy-to-interpret network
- Computational resources are limited (most complex algorithm)
- Data is very noisy (may infer spurious median vectors)

## Conclusion

- **MST**: Simplest, tree-based, no reticulations
- **MSN**: Simple, shows alternatives, minimal reticulations
- **TCS**: Statistically justified, may be disconnected
- **MJN**: Most complex, infers ancestors, highly reticulate

The choice depends on your data characteristics and research questions!